<a href="https://colab.research.google.com/github/antreev-brar/ML-projects/blob/master/textclassifier_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
!pip install tensorflow-hub
!pip install tfds-nightly

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("version--",tf.__version__)
print("hub-version:",hub.__version__)
print("GPU is ","avialable" if tf.config.experimental.list_physical_devices("GPU") else "not avialable")

version-- 2.2.0-rc2
hub-version: 0.8.0
GPU is  avialable


In [0]:
train_data , validation_data , test_data = tfds.load(
    name = "imdb_reviews",
    split = ('train[:60%]','train[60%:]','test'),
    as_supervised = True

)

In [5]:
train_example_batch , train_label_batch = next(iter(train_data.batch(10)))
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_label_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding , input_shape=[] ,dtype=tf.string , 
                           trainable = True)
hub_layer (train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [8]:
model= tf.keras.Sequential()
model.add(hub_layer)

model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
    optimizer = 'adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits= True),
    metrics=['accuracy']
)

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    validation_data= validation_data.batch(512),
                    epochs = 20, verbose =1
                    )

Epoch 1/20
30/30 [==============================] - 2s 82ms/step - loss: 0.7684 - accuracy: 0.5151 - val_loss: 0.7068 - val_accuracy: 0.5568
Epoch 2/20
30/30 [==============================] - 2s 79ms/step - loss: 0.6675 - accuracy: 0.5767 - val_loss: 0.6451 - val_accuracy: 0.6028
Epoch 3/20
30/30 [==============================] - 2s 77ms/step - loss: 0.6152 - accuracy: 0.6316 - val_loss: 0.5976 - val_accuracy: 0.6570
Epoch 4/20
30/30 [==============================] - 2s 78ms/step - loss: 0.5631 - accuracy: 0.6745 - val_loss: 0.5456 - val_accuracy: 0.7159
Epoch 5/20
30/30 [==============================] - 2s 79ms/step - loss: 0.5166 - accuracy: 0.7318 - val_loss: 0.5110 - val_accuracy: 0.7544
Epoch 6/20
30/30 [==============================] - 2s 79ms/step - loss: 0.4755 - accuracy: 0.7639 - val_loss: 0.4756 - val_accuracy: 0.7625
Epoch 7/20
30/30 [==============================] - 2s 79ms/step - loss: 0.4418 - accuracy: 0.7881 - val_loss: 0.4484 - val_accuracy: 0.7882
Epoch 8/20
30

In [34]:
result = model.evaluate(test_data.batch(10000) , verbose =1 )
for name, value in zip(model.metrics_names, result):
  print("%s: %.3f" % (name, value))

3/3 [==============================] - 2s 589ms/step - loss: 0.3347 - accuracy: 0.8534
loss: 0.335
accuracy: 0.853


In [18]:
test_data[0]

TypeError: ignored

In [37]:
test_batch , label_batch = next(iter(test_data.batch(100)))
test_batch[5]

<tf.Tensor: shape=(), dtype=string, numpy=b"My God, Ryan Gosling has made a lot of deep characters in his career, this is one of his wonderful acting jobs. For me this is a very deep movie, needs a lot of concentration, not because is difficult to watch, just because you understand it if you put your shoes in this kid, even though has everything and has famous father that is a writer, has a deeper mind, you don't understand why he kills this poor kid, until you really heard what he has to say and you start to think, at least to me, that a lot of things that he says is true. Simple kid, sweet, very gentle, in a way normal like any teenage, but inside of him suffer because he start to look at the world in a different way, then you understand why he did what he did. I recommend this movie for those who likes deep drama.">

In [35]:
#for (i in range(10):
  print(len(result))

2


IndexError: ignored

In [38]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3332 - accuracy: 0.8534
loss: 0.333
accuracy: 0.853
